In [1]:
import ipywidgets as widgets
import h5py
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
fileName_new = 'I:/1005.091 RVT PG&E/TimeHistoryDatabase/Database.hdf5'
fn = h5py.File(fileName_new,'r')
earthqaukes = list(fn.keys())

In [3]:
site_catalog = pd.read_csv('Site_catalog.csv')

In [4]:
# Date
# source latitude
# source longitude
# site latitide
# site longitude
# M(in bold)
# Rrup
# Rjb
# Repi
# Vs30
w = widgets.Text(
    value='',
    placeholder='constraint1: value1-value2,constraint2: value1-value2',
    description='Search criteria:',
    disabled=False
)
z = widgets.Text(
    value='',
    placeholder='columns',
    description='CSV fields:',
    disabled=False
)
EW1 = widgets.ToggleButton(
    value=False,
    description='EW1',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
NS1 = widgets.ToggleButton(
    value=False,
    description='NS1',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

EW2 = widgets.ToggleButton(
    value=False,
    description='EW2',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

NS2 = widgets.ToggleButton(
    value=False,
    description='NS2',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)


In [5]:
print('Available attributes of ground motions:')
stations = list(fn[earthqaukes[5458]].keys())
available_attributes = list(fn[earthqaukes[5458]][stations[0]]['EW1']['processed'].attrs.keys())# Here I choose a ground motion and get all the information
print(len(available_attributes))
for key in available_attributes:
    print(key)
display(w)
display(z)
display(EW1)
display(NS1)
display(EW2)
display(NS2)

Available attributes of ground motions:
55
recordDate
originTime
direction
durationTime
originDate
lastCorrectionDate
scaleFactor
samplingFreq
lon
lat
mag
depth
stationCode
stationLat
stationLon
snr
fLow
fHigh
pga
Ia
D5_75
D5_75_sorted
D20_80
repi_literature
rhypo_literature
rjb_literature
rrup_literature
rx_literature
magnitude_literature
ry_literature
ry0_literature
U_literature
T_literature
name_literature
Ztor_literature
Origin_day_
Origin_Time_
Latitude_
Longitude_
JMA_Depth_
JMA_Magnitude_
Region_Name_
Strike_
Dip_
Rake_
Mo_
MT_Depth_
MT_Magnitude_
Var_Red_
Unit_
Number_of_Stations_
Tectonic_Zhoa_
Tectonic_Garcia_
Focal_mechanism_Garcia_
Focal_mechanism_kegan_


Text(value='', description='Search criteria:', placeholder='constraint1: value1-value2,constraint2: value1-val…

Text(value='', description='CSV fields:', placeholder='columns')

ToggleButton(value=False, description='EW1', icon='check', tooltip='Description')

ToggleButton(value=False, description='NS1', icon='check', tooltip='Description')

ToggleButton(value=False, description='EW2', icon='check', tooltip='Description')

ToggleButton(value=False, description='NS2', icon='check', tooltip='Description')

In [6]:
directions = []
direction_dependant = ['snr', 'fLow', 'fHigh', 'pga', 'Ia', 'D5_75', 'D5_75_sorted', 'D20_80']
if EW1.value:
    directions.append('EW1')
if NS1.value:
    directions.append('NS1')
if EW2.value:
    directions.append('EW2')
if NS2.value:
    directions.append('NS2')
directions

['EW1', 'NS1', 'EW2', 'NS2']

In [7]:
constraints = w.value
columns = z.value.split(",")

In [8]:
constraints = constraints.split(",")

In [9]:
constraint_dic = {}

for constraint in constraints:
    print(constraint)
    constraint = constraint.split(":")
    name = constraint[0]
    value = constraint[1]
    value = value.split("-")
    value_float = [float(value[0]),float(value[1])]
    constraint_dic[name] = value_float

snr: 0.6-1.1


In [10]:
constraint
constraint_dic

{'snr': [0.6, 1.1]}

In [11]:
import numpy


In [12]:
my_str = 'Record code'
for column in columns:
    if column in direction_dependant:
        for direction in directions:
            my_str = my_str + ',' +column+'_'+direction 
    else:
        my_str = my_str + ',' +column
my_str

'Record code,lon,lat,stationLat,stationLon,pga_EW1,pga_NS1,pga_EW2,pga_NS2,Ia_EW1,Ia_NS1,Ia_EW2,Ia_NS2,D5_75_EW1,D5_75_NS1,D5_75_EW2,D5_75_NS2,D5_75_sorted_EW1,D5_75_sorted_NS1,D5_75_sorted_EW2,D5_75_sorted_NS2,D20_80_EW1,D20_80_NS1,D20_80_EW2,D20_80_NS2,rrup_literature,rrup_0,rrup_1,Ztor_literature,Ztor_0,Ztor_1,magnitude_literature,name_literature,Focal_mechanism_kegan_,Tectonic_Garcia_,snr_EW1,snr_NS1,snr_EW2,snr_NS2'

In [13]:
file = open('Arias_Intensity.csv','w+')
file.write(my_str+'\n') 
for earthquake in earthqaukes:
    g = fn[earthquake]
    stations = list(g.keys())
    for station in stations:
        try:
            GM_keys = list(g[station]['EW1']['processed'].attrs.keys())
        except KeyError:
            print(station + ' is corrupted')
            continue
        Have_conditions = 1
        for contraint_key in list(constraint_dic.keys()):
            if contraint_key in GM_keys:
                if ~(constraint_dic[contraint_key][0]<g[station]['EW1']['processed'].attrs[contraint_key] and
                    constraint_dic[contraint_key][1]>g[station]['EW1']['processed'].attrs[contraint_key]):
                    Have_conditions = 0 
                    break
            else:
                Have_conditions = 0
                break
        if Have_conditions:
            my_str = str(station)
            for column in columns:
                if column in direction_dependant:
                    for direction in directions:
                        try:
                            a = g[station][direction]['processed'].attrs[column]
                        except KeyError:
                            a = 'NAN'
                        if isinstance(a,str):
                            a = a
                        elif isinstance(a,float) or isinstance(a,numpy.int64):
                            a = str(a)
                        elif isinstance(a,numpy.ndarray):
                            a = a[0]
                            if isinstance(a,float):
                                a = str(a)
                            else:
                                a = a.decode()
                        my_str = my_str + ',' + a
                elif column in site_catalog:
                    try:
                        a = site_catalog[site_catalog['Station code'].str.contains(station[:6])].iloc[0][column]
                        a = str(a)
                    except IndexError:
                        a = 'NAN'
                    my_str = my_str + ',' + a

                else:
                    try:
                        a = g[station]['EW1']['processed'].attrs[column]

                    except KeyError:
                        a = 'NAN'
                    if isinstance(a,str):
                        a = a
                    elif isinstance(a,float) or isinstance(a,numpy.int64):
                        a = str(a)
                    elif isinstance(a,numpy.ndarray):
                        a = a[0]
                        if isinstance(a,float):
                            a = str(a)
                        else:
                            a = a.decode()
                    my_str = my_str + ',' + a


                
            file.write(my_str+'\n')
file.close()
#fn.close()

In [ ]:
fn.close()

In [40]:
earthqaukes[5458]

'20110311144600'

In [ ]:
site_catalog